# Accessing Open Cropland and landcover Maps

The following provides a simple example for how to access and view the cropland and landcover maps. 

To access these data you need to have an AWS account. 

In [1]:
import boto3
import geopandas as gpd
import folium
from shapely.geometry import Polygon
import json
from IPython.display import display

## Datasets

Start by getting a list of datasets in the bucket. 

In [12]:
session = boto3.Session()
s3_client = session.client('s3')
bucket_name = 'mappingafrica'
response = s3_client.list_objects_v2(Bucket=bucket_name)

objects = [obj["Key"] for obj in response["Contents"]]
objects

['croplands/geoparquet/congo_2022_v1.parquet',
 'croplands/pmtiles/congo_2022_v1.pmtiles',
 'croplands/pmtiles/zambia_2022_v1.pmtiles',
 'landcover/tanzania_2019.tif']

### Croplands

The croplands prefix contains field boundary maps for several African countries, stored as pmtiles (for a web-mapping application) and geoparquet (for analytics). We'l 

Read the GeoParquet data into a GeoDataFrame

In [2]:
s3_url = 's3://mappingafrica/croplands/geoparquet/congo_2022_v1.parquet'

gdf = gpd.read_parquet(s3_url)
gdf = gdf.dropna(subset=['geometry'])
gdf.head()

,id,tile,aoi,grp,geometry
0,1,664360,1.0,1.0,"MULTIPOLYGON (((17.45164 3.65598, 17.45174 3.6..."
1,2,664360,1.0,1.0,"MULTIPOLYGON (((17.44139 3.65297, 17.44139 3.6..."
2,3,664360,1.0,1.0,"MULTIPOLYGON (((17.44124 3.65281, 17.44132 3.6..."
3,4,664360,1.0,1.0,"MULTIPOLYGON (((17.44477 3.65036, 17.44487 3.6..."
4,5,664360,1.0,1.0,"MULTIPOLYGON (((17.44759 3.64945, 17.44757 3.6..."


Display the cropland map of 2022 for Congo

In [ ]:
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True) 
bounds = gdf.total_bounds

center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

m = folium.Map(location=[center_lat, center_lon], zoom_start=6, 
               tiles='cartodbpositron')

folium.GeoJson(gdf).add_to(m)

m.save('interactive_map.html')
m